In [1]:
# Import dependencies
import os
import csv
import pandas as pd
import numpy as np
import skfeature.function.sparse_learning_based.MCFS as MCFS
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from keras.utils import to_categorical
from scipy import sparse
import pickle

C:\Users\jnovic\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# process the data
data = pd.read_csv("../processed_data.csv")

In [3]:
data = data.drop("Unnamed: 0", axis = 1)

In [4]:
data.columns

Index(['SEQN', 'CBD120', 'CBD130', 'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040',
       'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090', 'DUQ211', 'DUQ240',
       'FSQ012', 'FSQ162', 'HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031D',
       'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'OCQ180', 'OCD270',
       'PAD645', 'PAQ655', 'PAD660', 'PAQ665', 'SLD010H', 'SMQ040', 'SMD460',
       'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDEDUC3', 'DMDMARTL', 'RIDEXPRG',
       'DMDHHSIZ', 'INDHHIN2', 'INDFMPIR', 'DRQSDIET', 'DRQSDT1', 'BMXWT',
       'BMXBMI', 'SLQ050'],
      dtype='object')

In [5]:
data["SLQ050"].value_counts()

2.0    4880
1.0    1582
0.0       2
Name: SLQ050, dtype: int64

In [6]:
data = data.loc[data["SLQ050"]!= 0.0,:]

In [7]:
X = data.drop(["SLQ050", "SEQN"], axis = 1)
y = data["SLQ050"]


In [8]:
y.value_counts()

2.0    4880
1.0    1582
Name: SLQ050, dtype: int64

In [9]:
# Step 1: Label-encode y data 
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [10]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0


Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


...

Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------


Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0


Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


In [11]:
X.head()


,CBD120,CBD130,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,...,DMDEDUC3,DMDMARTL,RIDEXPRG,DMDHHSIZ,INDHHIN2,INDFMPIR,DRQSDIET,DRQSDT1,BMXWT,BMXBMI
0,"(-2.142, 535.5]","(-1.028, 257.0]",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,"(1.5, 3.0]",4.0,"(-0.005, 1.25]",2.0,0.0,"(55.65, 111.3]","(20.725, 41.45]"
1,"(-2.142, 535.5]","(-1.028, 257.0]",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,"(3.0, 4.5]",0.0,"(1.25, 2.5]",2.0,0.0,"(55.65, 111.3]","(20.725, 41.45]"
2,"(-2.142, 535.5]","(-1.028, 257.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,"(1.5, 3.0]",10.0,"(3.75, 5.0]",1.0,0.0,"(55.65, 111.3]","(20.725, 41.45]"
3,"(-2.142, 535.5]","(-1.028, 257.0]",2.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,...,0.0,1.0,0.0,"(1.5, 3.0]",15.0,"(3.75, 5.0]",1.0,0.0,"(-0.223, 55.65]","(-0.0829, 20.725]"
4,"(-2.142, 535.5]","(-1.028, 257.0]",3.0,3.0,3.0,3.0,3.0,1.0,2.0,1.0,...,0.0,3.0,0.0,"(-0.006, 1.5]",0.0,"(3.75, 5.0]",2.0,0.0,"(55.65, 111.3]","(41.45, 62.175]"


In [12]:
#lable encode the x datat
for col in X:
    k =label_encoder.fit(X[col])
    X[col] = label_encoder.transform(X[col])
    print(col)

CBD120
CBD130
DPQ010
DPQ020
DPQ030
DPQ040
DPQ050
DPQ060
DPQ070
DPQ080
DPQ090
DUQ211
DUQ240
FSQ012
FSQ162
HIQ011
HIQ031A
HIQ031B
HIQ031D
HIQ031F
HIQ031G
HIQ031H
HIQ031I
OCQ180
OCD270
PAD645
PAQ655
PAD660
PAQ665
SLD010H
SMQ040
SMD460
RIAGENDR
RIDAGEYR
RIDRETH1
DMDEDUC3
DMDMARTL
RIDEXPRG
DMDHHSIZ
INDHHIN2
INDFMPIR
DRQSDIET
DRQSDT1
BMXWT
BMXBMI


In [13]:
X.head()

,CBD120,CBD130,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,...,DMDEDUC3,DMDMARTL,RIDEXPRG,DMDHHSIZ,INDHHIN2,INDFMPIR,DRQSDIET,DRQSDT1,BMXWT,BMXBMI
0,0,0,1,0,0,0,0,0,0,0,...,0,4,0,1,4,0,2,0,3,1
1,0,0,2,0,0,0,0,0,0,0,...,0,1,0,2,0,1,2,0,3,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,8,3,1,0,3,1
3,0,0,2,1,0,3,3,0,0,0,...,0,1,0,1,12,3,1,0,0,0
4,0,0,3,3,3,3,3,1,2,1,...,0,3,0,0,0,3,2,0,3,2


In [14]:
#convert to a sparse matrix
Xsp =sparse.coo_matrix(X )


In [15]:
#select 15 features
feat_sel = SelectKBest(k=15)

In [16]:
step1 = feat_sel.fit(Xsp, encoded_y)

In [17]:
step1.get_support(indices=True)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 15, 17, 18, 23, 30, 33, 35],
      dtype=int64)

In [37]:
#transform X to only the 15 features
X= step1.transform(Xsp)

In [38]:
#split into test-train-validate

X_train, X_tv, y_train, y_tv = train_test_split(X, encoded_y, train_size=.7, random_state = 42 )

C:\Users\jnovic\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [39]:
X_test, X_val, y_test, y_val = train_test_split(X_tv, y_tv, train_size = .7, random_state = 42)

C:\Users\jnovic\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [40]:
# models
model1 = LogisticRegression()

In [41]:
model1 = model1.fit(X_train, y_train)

In [42]:
predict = model1.predict(X_test)

In [43]:
print(model1.score(X_train, y_train))

0.7800132655317268


In [44]:
print(model1.score(X_test, y_test))

0.7840825350036846


In [45]:
print(classification_report(y_test, predict))

             precision    recall  f1-score   support

          0       0.69      0.25      0.37       340
          1       0.79      0.96      0.87      1017

avg / total       0.77      0.78      0.74      1357



In [46]:
model2 = SVC(kernel="rbf")

In [47]:
model2 = model2.fit(X_train, y_train)

In [48]:
predict2 = model2.predict(X_test)

In [49]:
print(classification_report(y_test, predict2))

             precision    recall  f1-score   support

          0       0.64      0.21      0.32       340
          1       0.78      0.96      0.86      1017

avg / total       0.75      0.77      0.73      1357



In [52]:
#Save model
filename = 'LogRegression.sav'
pickle.dump(model1, open(filename, 'wb'))

In [55]:
test = data.drop(["SLQ050", "SEQN"], axis = 1)

In [56]:
step1.get_support(indices=True)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 15, 17, 18, 23, 30, 33, 35],
      dtype=int64)

In [61]:
data.iloc[:,2:10
         ]

,CBD130,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070
0,"(-1.028, 257.0]",1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(-1.028, 257.0]",2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(-1.028, 257.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(-1.028, 257.0]",2.0,1.0,0.0,3.0,3.0,0.0,0.0
4,"(-1.028, 257.0]",3.0,3.0,3.0,3.0,3.0,1.0,2.0
5,"(-1.028, 257.0]",0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,"(-1.028, 257.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"(-1.028, 257.0]",0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,"(-1.028, 257.0]",1.0,1.0,0.0,1.0,0.0,0.0,0.0
9,"(-1.028, 257.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0


<6462x15 sparse matrix of type '<class 'numpy.int64'>'
	with 31157 stored elements in Compressed Sparse Row format>